In [50]:
import pandas as pd
import numpy as np

In [51]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [154]:
train = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/train.csv")
test = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/test.csv")

In [155]:
train['date'] = pd.to_datetime(train["date"])
train['day_of_week'] = train['date'].dt.day_of_week
train['day_of_year'] = train['date'].dt.dayofyear
train['day_of_month'] = train['date'].dt.day
train['week'] = train['date'].dt.isocalendar().week.astype('int32')
train['month'] = train['date'].dt.month
train['quarter'] = train['date'].dt.quarter
train['year'] = train['date'].dt.year
train['date_ordinal'] = train['date'].apply(lambda x: x.toordinal())

In [156]:
test['date'] = pd.to_datetime(test["date"])
test['day_of_week'] = test['date'].dt.day_of_week
test['day_of_year'] = test['date'].dt.dayofyear
test['day_of_month'] = test['date'].dt.day
test['week'] = test['date'].dt.isocalendar().week.astype('int32')
test['month'] = test['date'].dt.month
test['quarter'] = test['date'].dt.quarter
test['year'] = test['date'].dt.year
test['date_ordinal'] = test['date'].apply(lambda x: x.toordinal())

In [157]:
def calculate_pay_date_distance(date):
    day_of_month = date.day
    last_day = date.days_in_month
    
    if day_of_month == last_day:
        return 0
    elif day_of_month < 15:
        return day_of_month
    else:
        return day_of_month - 15

train['num_from_pay_date'] = train['date'].apply(calculate_pay_date_distance)
test['num_from_pay_date'] = test['date'].apply(calculate_pay_date_distance)

In [159]:
train.shape
# test.shape

(3000888, 15)

In [56]:
train.head()

,id,date,store_nbr,family,sales,onpromotion,day_of_week,day_of_year,day_of_month,week,month,quarter,year,date_ordinal,num_from_pay_date
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,1,1,1,1,1,1,2013,734869,1
1,1,2013-01-01,1,BABY CARE,0.0,0,1,1,1,1,1,1,2013,734869,1
2,2,2013-01-01,1,BEAUTY,0.0,0,1,1,1,1,1,1,2013,734869,1
3,3,2013-01-01,1,BEVERAGES,0.0,0,1,1,1,1,1,1,2013,734869,1
4,4,2013-01-01,1,BOOKS,0.0,0,1,1,1,1,1,1,2013,734869,1


In [161]:
test_drop = test.drop(columns=['date'])
test_drop = pd.get_dummies(test_drop, columns=['family'], dtype=int)

In [162]:
test_drop.shape

(28512, 45)

In [57]:
train_drop = train.drop(columns=['id', 'date'])
train_drop = pd.get_dummies(train_drop, columns=['family'], dtype=int)

In [58]:
train_drop.head()

,store_nbr,sales,onpromotion,day_of_week,day_of_year,day_of_month,week,month,quarter,year,...,family_MAGAZINES,family_MEATS,family_PERSONAL CARE,family_PET SUPPLIES,family_PLAYERS AND ELECTRONICS,family_POULTRY,family_PREPARED FOODS,family_PRODUCE,family_SCHOOL AND OFFICE SUPPLIES,family_SEAFOOD
0,1,0.0,0,1,1,1,1,1,1,2013,...,0,0,0,0,0,0,0,0,0,0
1,1,0.0,0,1,1,1,1,1,1,2013,...,0,0,0,0,0,0,0,0,0,0
2,1,0.0,0,1,1,1,1,1,1,2013,...,0,0,0,0,0,0,0,0,0,0
3,1,0.0,0,1,1,1,1,1,1,2013,...,0,0,0,0,0,0,0,0,0,0
4,1,0.0,0,1,1,1,1,1,1,2013,...,0,0,0,0,0,0,0,0,0,0


In [164]:
train_drop.columns

Index(['store_nbr', 'sales', 'onpromotion', 'day_of_week', 'day_of_year',
       'day_of_month', 'week', 'month', 'quarter', 'year', 'date_ordinal',
       'num_from_pay_date', 'family_AUTOMOTIVE', 'family_BABY CARE',
       'family_BEAUTY', 'family_BEVERAGES', 'family_BOOKS',
       'family_BREAD/BAKERY', 'family_CELEBRATION', 'family_CLEANING',
       'family_DAIRY', 'family_DELI', 'family_EGGS', 'family_FROZEN FOODS',
       'family_GROCERY I', 'family_GROCERY II', 'family_HARDWARE',
       'family_HOME AND KITCHEN I', 'family_HOME AND KITCHEN II',
       'family_HOME APPLIANCES', 'family_HOME CARE', 'family_LADIESWEAR',
       'family_LAWN AND GARDEN', 'family_LINGERIE', 'family_LIQUOR,WINE,BEER',
       'family_MAGAZINES', 'family_MEATS', 'family_PERSONAL CARE',
       'family_PET SUPPLIES', 'family_PLAYERS AND ELECTRONICS',
       'family_POULTRY', 'family_PREPARED FOODS', 'family_PRODUCE',
       'family_SCHOOL AND OFFICE SUPPLIES', 'family_SEAFOOD'],
      dtype='object')

In [166]:
test_drop_scaled = x_scaler.transform(test_drop.drop(columns=['id']))

In [167]:
test_drop_scaled.shape

(28512, 44)

In [173]:
predictions = model_prepu(torch.tensor(test_drop_scaled, dtype=torch.float32).to(device))

In [176]:
predictions.shape

torch.Size([28512, 1])

In [177]:
predictions_np = predictions.detach().cpu().numpy()

In [180]:
print(predictions_np.flatten())

[3.5505753e+00 4.1702837e-03 4.2531610e+00 ... 1.0682756e+03 1.2298236e+02
 1.0840978e+01]


In [181]:
save_test_pd = test_drop[['id']].copy()

In [183]:
save_test_pd['sales'] = predictions_np.flatten()

In [184]:
save_test_pd.head()

,id,sales
0,3000888,3.550575
1,3000889,0.004170
2,3000890,4.253161
3,3000891,2224.120117
4,3000892,0.082840


In [185]:
save_test_pd.to_csv('submission.csv', index=False)

In [60]:
def custom_activation(x, scale):
    x_relu = torch.relu(x)
    power = F.softsign(scale) + 1
    return torch.pow(x_relu, power)

class CustomActivationLayer(nn.Module):
    def __init__(self, num_features):
        super(CustomActivationLayer, self).__init__()
        self.scale = nn.Parameter(torch.zeros(num_features))

    def forward(self, inputs):
        return custom_activation(inputs, self.scale)

In [97]:
class FeedFowardPRePU(nn.Module):
    def __init__(self):
        super(FeedFowardPRePU, self).__init__()
        self.layers = nn.Sequential(
#             nn.Linear(44, 256),
#             nn.ReLU(),
#             nn.Linear(256, 256),
#             nn.ReLU(),
#             nn.Linear(256, 256),
#             nn.ReLU(),
#             nn.Linear(256, 1)

            nn.Linear(44, 1024),
            CustomActivationLayer(1024),
            nn.Linear(1024, 1024),
            CustomActivationLayer(1024),
            nn.Linear(1024, 1024),
            CustomActivationLayer(1024),
            nn.Linear(1024, 1024),
            CustomActivationLayer(1024),
            nn.Linear(1024, 1024),
            CustomActivationLayer(1024),
            nn.Linear(1024, 1024),
            CustomActivationLayer(1024),
            nn.Linear(1024, 1024),
            CustomActivationLayer(1024),
            nn.Linear(1024, 1024),
            CustomActivationLayer(1024),
            nn.Linear(1024, 1024),
            CustomActivationLayer(1024),
            nn.Linear(1024, 1)
        )

    def forward(self, x):
        return self.layers(x)
    
model_prepu = FeedFowardPRePU().to(device)
summary(model_prepu, input_size=(1, 44))

Layer (type:depth-idx)                   Output Shape              Param #
FeedFowardPRePU                          [1, 1]                    --
├─Sequential: 1-1                        [1, 1]                    --
│    └─Linear: 2-1                       [1, 1024]                 46,080
│    └─CustomActivationLayer: 2-2        [1, 1024]                 1,024
│    └─Linear: 2-3                       [1, 1024]                 1,049,600
│    └─CustomActivationLayer: 2-4        [1, 1024]                 1,024
│    └─Linear: 2-5                       [1, 1024]                 1,049,600
│    └─CustomActivationLayer: 2-6        [1, 1024]                 1,024
│    └─Linear: 2-7                       [1, 1024]                 1,049,600
│    └─CustomActivationLayer: 2-8        [1, 1024]                 1,024
│    └─Linear: 2-9                       [1, 1024]                 1,049,600
│    └─CustomActivationLayer: 2-10       [1, 1024]                 1,024
│    └─Linear: 2-11                   

In [98]:
x = train_drop.drop('sales', axis=1).astype('float32')
y = train_drop['sales']

x.shape

(3000888, 44)

In [99]:
from sklearn.preprocessing import StandardScaler
x_scaler = StandardScaler()
x_train_scaled = x_scaler.fit_transform(x)
x_train_scaled.shape

(3000888, 44)

In [141]:
class CustomDataLoader:
    def __init__(self, features, labels, batch_size=1, validation_size=0.0, shuffle=False):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        if validation_size > 0:
            train_data, val_data, train_labels, val_labels = train_test_split(
                features, labels, test_size=validation_size, random_state=42
            )
            self.train_loader = DataLoader(
                TensorDataset(
                    torch.tensor(train_data).float().to(device), 
                    torch.tensor(train_labels).float().to(device),
                ),
                batch_size=batch_size,
                shuffle=shuffle,
#                 pin_memory=True
            )
            self.val_loader = DataLoader(
                TensorDataset(
                    torch.tensor(val_data).float().to(device), 
                    torch.tensor(val_labels).float().to(device),
                ),
                batch_size=batch_size,
                shuffle=shuffle,
#                 pin_memory=True
            )
        else:
            self.train_loader = DataLoader(
                TensorDataset(
                    torch.tensor(features).float().to(device), 
                    torch.tensor(labels).float().to(device),
                ),
                batch_size=batch_size,
                shuffle=shuffle,
#                 pin_memory=True
            )
            self.val_loader = None

    def get_train_loader(self):
        return self.train_loader

    def get_val_loader(self):
        return self.val_loader

def evaluate_model(model, custom_train_loader, criterion, optimizer, device):
    num_epochs = 100
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        index = 0
        for inputs, labels in custom_train_loader.get_train_loader():
#             inputs = inputs.to(device, non_blocking=True)
#             labels = labels.to(device, non_blocking=True)
            optimizer.zero_grad()
#             inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)

            outputs = model(inputs.view(-1, 44))
            loss = criterion(outputs, labels.view(-1, 1))
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
#             torch.nn.utils.clip_grad_value_(model.parameters(), clip_value=100)

            optimizer.step()

            running_loss += loss.item()
            index += 1
            current_loss = running_loss / index
            
            print(f"Epoch {epoch+1:4d} | Batch {index}/{len(custom_train_loader.get_train_loader())} | Train Loss: {current_loss:.4f}", end='\r')

        avg_train_loss = running_loss / len(custom_train_loader.get_train_loader())

#         model.eval()
#         running_val_loss = 0.0
#         with torch.no_grad():
#             for inputs, labels in custom_train_loader.get_val_loader():
#                 outputs = model(inputs.view(-1, 44))
#                 val_loss = criterion(outputs, labels.view(-1, 1))
#                 running_val_loss += val_loss.item()

#         avg_val_loss = running_val_loss / len(custom_train_loader.get_val_loader())
        print(f"Epoch {epoch+1:4d} | Avg Train Loss: {avg_train_loss:.4f}          ")
custom_train_loader = CustomDataLoader(x_train_scaled, y.values, batch_size=32768, validation_size=0, shuffle=True)

In [142]:
class MSLELoss(nn.Module):
    def __init__(self):
        super(MSLELoss, self).__init__()

    def forward(self, y_pred, y_true):
        # Ensure that predictions and targets are all positive and non-zero
        y_pred = torch.clamp(y_pred, min=1e-7)  # clamp to avoid log(0)
        y_true = torch.clamp(y_true, min=1e-7)  # clamp to avoid log(0)
        
        # Compute Mean Squared Logarithmic Error
        loss = torch.mean((torch.log1p(y_pred) - torch.log1p(y_true)) ** 2)
        return loss

In [186]:
torch.save(model_prepu, 'prepu_kaggle.pth')

In [143]:
print(device)
criterion = MSLELoss()
optimizer = optim.Adam(model_prepu.parameters())

cuda
Epoch    1 | Avg Train Loss: 0.5485          
Epoch    2 | Avg Train Loss: 0.3511          
Epoch    3 | Avg Train Loss: 0.3057          
Epoch    4 | Avg Train Loss: 0.2918          
Epoch    5 | Avg Train Loss: 0.2830          
Epoch    6 | Avg Train Loss: 0.2856          
Epoch    7 | Avg Train Loss: 0.2675          
Epoch    8 | Avg Train Loss: 0.2590          
Epoch    9 | Avg Train Loss: 0.2497          
Epoch   10 | Avg Train Loss: 0.2476          
Epoch   11 | Avg Train Loss: 0.2428          
Epoch   12 | Avg Train Loss: 0.2345          
Epoch   13 | Avg Train Loss: 0.2307          
Epoch   14 | Avg Train Loss: 0.2337          
Epoch   15 | Avg Train Loss: 0.2240          
Epoch   16 | Avg Train Loss: 0.2241          
Epoch   17 | Avg Train Loss: 0.2214          
Epoch   18 | Avg Train Loss: 0.2176          
Epoch   19 | Avg Train Loss: 0.2126          
Epoch   20 | Avg Train Loss: 0.2209          
Epoch   21 | Avg Train Loss: 0.2107          
Epoch   22 | Avg Train Loss: 

KeyboardInterrupt: 

In [ ]:
evaluate_model(model_prepu, custom_train_loader, criterion, optimizer, device=device)

In [96]:
print(device)
criterion = MSLELoss()
optimizer = optim.Adam(model_prepu.parameters())
evaluate_model(model_prepu, custom_train_loader, criterion, optimizer, device=device)

cuda
Epoch    1 | Train Loss:     5.3790
Epoch    2 | Train Loss:     2.1275


KeyboardInterrupt: 

In [81]:
print(device)
criterion = MSLELoss()
optimizer = optim.Adam(model_prepu.parameters())
evaluate_model(model_prepu, custom_train_loader, criterion, optimizer, device=device)

cuda



KeyboardInterrupt



import tensorflow as tf
def custom_activation(x, scale):
    x_relu = tf.math.maximum(x, 0)
    power = tf.nn.softsign(scale) * 0.5 + 1.5
    return tf.math.pow(x_relu, power)

class customActivationLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(customActivationLayer, self).__init__()

    def build(self, input_shape):
        self.scale = self.add_weight(shape=(input_shape[-1],), initializer="zeros", trainable=True)

    def call(self, inputs):
        return custom_activation(inputs, self.scale)

In [12]:
column_data_types = train_drop.dtypes
# print(column_data_types)
# print(model.input_shape)
# missing_values = x_train.isna().any().any()

# if missing_values:
#     print("There are missing values in x_train.")
# else:
#     print("There are no missing values in x_train.")


In [13]:
x = train_drop.drop('sales', axis=1).astype('float32')
y = train_drop['sales']
# print(x.shape)
# print(y.shape)

In [14]:
from sklearn.preprocessing import StandardScaler
x_scaler = StandardScaler()
x_train_scaled = x_scaler.fit_transform(x)

In [15]:
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler

# x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

# x_scaler = StandardScaler()
# x_train_scaled = x_scaler.fit_transform(x_train)
# x_val_scaled = x_scaler.transform(x_val)

# y_scaler = StandardScaler()
# y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1, 1))
# y_val_scaled = y_scaler.transform(y_val.values.reshape(-1, 1))

In [16]:
print("TensorFlow version:", tf.__version__)
print("Available GPU devices:", tf.config.list_physical_devices('GPU'))


TensorFlow version: 2.15.0
Available GPU devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [17]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(44,)),
    tf.keras.layers.Dense(512),
    customActivationLayer(),
    tf.keras.layers.Dense(512),
    customActivationLayer(),
    tf.keras.layers.Dense(512),
    customActivationLayer(),
    tf.keras.layers.Dense(512),
    customActivationLayer(),
    tf.keras.layers.Dense(512),
    customActivationLayer(),
    tf.keras.layers.Dense(512),
    customActivationLayer(),
    tf.keras.layers.Dense(512),
    customActivationLayer(),
    tf.keras.layers.Dense(512),
    customActivationLayer(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1)
])
model.compile(optimizer='adam', loss='mean_squared_logarithmic_error')
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 512)            │        23,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ custom_activation_layer         │ (None, 512)            │           512 │
│ (customActivationLayer)         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ custom_activation_layer_1       │ (None, 512)            │           512 │
│ (customActivationLayer)         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ custom_activation_layer_2       │ (None, 512)            │           512 │
│ (customActivationLayer)         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ custom_activation_layer_3       │ (None, 512)            │           512 │
│ (customActivationLayer)         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ custom_activation_layer_4       │ (None, 512)            │           512 │
│ (customActivationLayer)         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ custom_activation_layer_5       │ (None, 512)            │           512 │
│ (customActivationLayer)         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ custom_activation_layer_6       │ (None, 512)            │           512 │
│ (customActivationLayer)         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ custom_activation_layer_7       │ (None, 512)            │           512 │
│ (customActivationLayer)         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 512)            │       262,65

 Total params: 14,473,729 (55.21 MB)

 Trainable params: 14,473,729 (55.21 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
from tensorflow.keras.callbacks import Callback

class RMSLECallback(Callback):
    def __init__(self, x_val, y_val, scaler):
        super().__init__()
        self.x_val = x_val
        self.y_val = y_val
        self.scaler = scaler
    
    def rmsle(y_true, y_pred):
        # Add 1 to avoid taking log of zero
        log_true = np.log(y_true + 1)
        log_pred = np.log(y_pred + 1)
        squared_log_error = (log_true - log_pred) ** 2
        mean_squared_log_error = np.mean(squared_log_error)
        return np.sqrt(mean_squared_log_error)

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % 20 == 0:
            y_pred_scaled = self.model.predict(self.x_val)
            y_pred = self.scaler.inverse_transform(y_pred_scaled)
            y_val_rescaled = self.scaler.inverse_transform(self.y_val.reshape(-1, 1))

            rmsle_value = self.rmsle(y_val_rescaled.flatten(), y_pred.flatten())
            print(f"Epoch {epoch+1}: RMSLE = {rmsle_value:.4f}")
        else:
            print(f"Skipping RMSLE calculation for Epoch {epoch+1}")

In [19]:
history = model.fit(x_train_scaled, y, epochs=100, batch_size=32768)

Epoch 1/100


I0000 00:00:1713590250.112807     103 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1713590250.144793     103 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


91/92 ━━━━━━━━━━━━━━━━━━━━ 0s 398ms/step - loss: 15.8418

W0000 00:00:1713590292.653015     103 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


92/92 ━━━━━━━━━━━━━━━━━━━━ 63s 465ms/step - loss: 15.8415
Epoch 2/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 37s 396ms/step - loss: 15.8550
Epoch 3/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 37s 396ms/step - loss: 15.8349
Epoch 4/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 37s 396ms/step - loss: 15.8209
Epoch 5/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 37s 396ms/step - loss: 15.8125
Epoch 6/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 37s 396ms/step - loss: 15.8212
Epoch 7/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 37s 396ms/step - loss: 15.8347
Epoch 8/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 37s 396ms/step - loss: 15.8169
Epoch 9/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 37s 396ms/step - loss: 15.8235
Epoch 10/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 37s 396ms/step - loss: 15.8273
Epoch 11/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 37s 396ms/step - loss: 15.8343
Epoch 12/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 37s 396ms/step - loss: 15.8235
Epoch 13/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 37s 396ms/step - loss: 15.8226
Epoch 14/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 37s 396ms/step - loss: 15.8357
Epoch 15/100
92/92 ━━━━━━━━

In [ ]:
history = model.fit(x_train_scaled, y_train, epochs=100, batch_size=32768,validation_data=(x_val_scaled, y_val))

Epoch 1/1000
 1/74 ━━━━━━━━━━━━━━━━━━━━ 8:50 7s/step - loss: 15.7405

W0000 00:00:1713583645.780929      84 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - loss: 7.3307

W0000 00:00:1713583656.689420      85 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


74/74 ━━━━━━━━━━━━━━━━━━━━ 20s 177ms/step - loss: 7.2887 - val_loss: 1.9433
Epoch 2/1000
74/74 ━━━━━━━━━━━━━━━━━━━━ 9s 127ms/step - loss: 1.8692 - val_loss: 1.7679
Epoch 3/1000
74/74 ━━━━━━━━━━━━━━━━━━━━ 9s 127ms/step - loss: 1.6909 - val_loss: 1.4778
Epoch 4/1000
74/74 ━━━━━━━━━━━━━━━━━━━━ 9s 127ms/step - loss: 1.4583 - val_loss: 1.3794
Epoch 5/1000
74/74 ━━━━━━━━━━━━━━━━━━━━ 9s 127ms/step - loss: 1.3149 - val_loss: 1.2803
Epoch 6/1000
74/74 ━━━━━━━━━━━━━━━━━━━━ 9s 127ms/step - loss: 1.2470 - val_loss: 1.1757
Epoch 7/1000
74/74 ━━━━━━━━━━━━━━━━━━━━ 9s 127ms/step - loss: 1.1719 - val_loss: 1.0522
Epoch 8/1000
74/74 ━━━━━━━━━━━━━━━━━━━━ 9s 127ms/step - loss: 1.0500 - val_loss: 1.2555
Epoch 9/1000
74/74 ━━━━━━━━━━━━━━━━━━━━ 9s 127ms/step - loss: 1.0004 - val_loss: 0.8538
Epoch 10/1000
74/74 ━━━━━━━━━━━━━━━━━━━━ 9s 127ms/step - loss: 0.8286 - val_loss: 0.8037
Epoch 11/1000
74/74 ━━━━━━━━━━━━━━━━━━━━ 9s 127ms/step - loss: 0.7658 - val_loss: 0.8230
Epoch 12/1000
74/74 ━━━━━━━━━━━━━━━━━━━━ 1